In [13]:
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.visualization import plot_bloch_multivector
from qiskit.quantum_info import Statevector
from IPython.display import display
from qiskit.tools.visualization import plot_histogram

sim = Aer.get_backend("aer_simulator")

# Q10  前問の結果を測定した場合、0と1を測定する確率はそれぞれ理論的にはいくつになるか。また実際に測定して理論値と比較してみよ。

## 解答

In [30]:
# Bell状態をkey, Xゲートを作用させる量子ビットのidのリストをvalueとする辞書を作成
bell_state_to_init_state = {
    "(|00>+|11>)/sqrt(2)": [],
    "(|00>-|11>)/sqrt(2)": [0],
    "(|01>+|10>)/sqrt(2)": [1],
    "(|01>-|10>)/sqrt(2)": [0, 1]
}

for bell_state, init_state in bell_state_to_init_state.items():
    print(bell_state)
    qc = QuantumCircuit(2, 2)
    for i in init_state:
        qc.x(qubit=i)
    qc.h(qubit=0)
    qc.cx(control_qubit=0, target_qubit=1)
    state_vec = Statevector(data=qc).to_dict()

    for bitstring, wave_func in state_vec.items():
        # 係数の絶対値の二乗を計算
        prob = wave_func.real**2 + wave_func.imag**2
        # 理論的な確率をプリント
        print(f"Bitstring {bitstring}: "
              f"Expected observation prob. = {np.round(prob, decimals=8)}")

    qc.measure(qubit=[0, 1], cbit=[0, 1])
    shots = 100000
    counts = sim.run(circuits=qc, shots=shots).result().get_counts()
    for bitstring, count in counts.items():
        # 実測の確率をプリント
        print(f"Bitstring {bitstring}: "
              f"Actual observation prob. = {np.round(count/shots, decimals=2)}")

(|00>+|11>)/sqrt(2)
Bitstring 00: Expected observation prob. = 0.5
Bitstring 11: Expected observation prob. = 0.5
Bitstring 11: Actual observation prob. = 0.5
Bitstring 00: Actual observation prob. = 0.5
(|00>-|11>)/sqrt(2)
Bitstring 00: Expected observation prob. = 0.5
Bitstring 11: Expected observation prob. = 0.5
Bitstring 00: Actual observation prob. = 0.5
Bitstring 11: Actual observation prob. = 0.5
(|01>+|10>)/sqrt(2)
Bitstring 01: Expected observation prob. = 0.5
Bitstring 10: Expected observation prob. = 0.5
Bitstring 01: Actual observation prob. = 0.5
Bitstring 10: Actual observation prob. = 0.5
(|01>-|10>)/sqrt(2)
Bitstring 01: Expected observation prob. = 0.5
Bitstring 10: Expected observation prob. = 0.5
Bitstring 01: Actual observation prob. = 0.5
Bitstring 10: Actual observation prob. = 0.5


## 解説
基底状態すなわち$|0\rangle$および$|1\rangle$に掛かる係数の絶対値の二乗が、その基底状態の測定確率を表す。Bell状態の1つである$(|00\rangle+|11\rangle)/\sqrt{2}$を具体的に見てみると、$|00\rangle$と$|11\rangle$に掛かる係数は共通の$1/\sqrt{2}$である。故に、$|00\rangle$と$|11\rangle$が出力される確率は共通の$|1/\sqrt{2}|^2 = 0.5$であると理論的に予測できる。

これを実際に測定して確認するには、測定を複数回行い、$|00\rangle$が出力される回数および$|11\rangle$が出力される回数をカウントして、それらを測定の回数で割ればよい。測定の回数(=shots)が十分に大きければ、それぞれが0.5に収束することを確認できる。